In [1]:
import json
import pandas as pd
from pandas import DataFrame, Series
from elasticsearch import Elasticsearch

In [2]:
hosts = ['localhost'] # on MacOS this could be something else, e.g. ['192.168.99.100']
username = 'elastic'
password = 'changeme'
es = Elasticsearch(hosts=hosts, http_auth="{}:{}".format(username, password))
es.info()

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'Dvit14MhRgiEoTxfIkuzYA',
 'name': 'osb6xs4',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2017-04-28T17:43:27.229Z',
  'build_hash': '780f8c4',
  'build_snapshot': False,
  'lucene_version': '6.5.0',
  'number': '5.4.0'}}

In [6]:
!head data/komoot.json -n 1 | cut -c1-100

{"_embedded":{"tours":[{"status":"private","type":"tour_recorded","date":"2017-05-30T17:44:39.000+02


In [8]:
# If the index has been created before it can be removed by this line
# es.indices.delete(index='komoot')

{'acknowledged': True}

In [9]:
mapping = '''
{
  "mappings": {
    "tour": {
      "properties": {
        "start_point": {
          "type": "geo_point"
        }
      }
    }
  }
}'''
es.indices.create(index='komoot', ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True}

In [12]:
# Read data into python dictionary
# You can use `data/komoot-sample-data.json` in case you have no other data available.
with open('data/komoot.json') as data_file:    
    data = json.load(data_file)

In [13]:
df = DataFrame(data['_embedded']['tours'])
df['start_point'] = df['start_point']\
    .map(lambda item: [item['lng'], item['lat']])
df.head()

,_embedded,_links,changed_at,constitution,date,difficulty,distance,duration,elevation_down,elevation_up,...,map_image_preview,name,path,query,sport,start_point,status,summary,time_in_motion,type
0,"{'creator': {'display_name': 'Fabian Müller', ...",{'creator': {'href': 'http://api.komoot.de/v00...,2017-05-30T18:06:11.000Z,NaN,2017-05-30T17:44:39.000+02:00,NaN,18743.235558,2923,265.501410,272.189527,...,{'src': 'https://photos.komoot.de/www/maps/173...,Die letzten Sonnenstrahlen vor dem Regen genießen,NaN,NaN,touringbicycle,"[8.852355, 48.797043]",private,NaN,2793.0,tour_recorded
1,"{'creator': {'display_name': 'Fabian Müller', ...",{'creator': {'href': 'http://api.komoot.de/v00...,2017-05-27T15:45:02.000Z,NaN,2017-05-27T16:49:11.000+02:00,NaN,19751.410759,2952,252.190878,214.774036,...,{'src': 'https://photos.komoot.de/www/maps/171...,"Sommer, Sonne, Sonnenschein",NaN,NaN,touringbicycle,"[8.859056, 48.8203]",public,NaN,2867.0,tour_recorded
2,"{'creator': {'display_name': 'Fabian Müller', ...",{'creator': {'href': 'http://api.komoot.de/v00...,2017-05-26T19:02:26.000Z,NaN,2017-05-26T12:31:37.000+02:00,NaN,59379.904806,16155,423.229605,413.177929,...,{'src': 'https://photos.komoot.de/www/maps/171...,Am Lech entlang,NaN,NaN,mtb,"[10.891965, 48.3165]",public,NaN,12277.0,tour_recorded
3,"{'creator': {'display_name': 'Fabian Müller', ...",{'creator': {'href': 'http://api.komoot.de/v00...,2017-05-26T20:10:03.000Z,NaN,2017-05-25T11:53:12.000+02:00,NaN,89096.735223,23138,623.187553,613.072361,...,{'src': 'https://photos.komoot.de/www/maps/170...,Sunshine Augsburg,NaN,NaN,touringbicycle,"[10.8807535, 48.31936]",public,NaN,15843.0,tour_recorded
4,"{'creator': {'display_name': 'Fabian Müller', ...",{'creator': {'href': 'http://api.komoot.de/v00...,2017-05-23T13:49:48.000Z,NaN,2017-05-23T14:10:04.000+02:00,NaN,31262.654622,5753,368.245151,375.785043,...,{'src': 'https://photos.komoot.de/www/maps/169...,Tour 23.05.2017 15:46,NaN,NaN,touringbicycle,"[8.857647, 48.802353]",public,NaN,5393.0,tour_recorded


In [ ]:
for i, row in df.iterrows():
    res = es.index(index='komoot', doc_type='tour', id=row['id'], body=row.to_json())
    print(row['id'], res['result'])